In [1]:
#Author Lana Chen(m073040105@g-mail.nsysu.edu.tw)
#Update: Obt,10th,2019
#Transform news to word embedding, and store them in ./embedding/output.npy

In [2]:
import pandas as pd
import re
from shutil import copyfile
import numpy as np

In [3]:
#read file(It's important that the input file must look like the following: )
df_read= pd.read_csv('myinput.csv') #your news name
df_read

,context
0,〔記者林宥辰／綜合報導〕備戰台灣大賽，中信兄弟昨、今兩天都舉辦隊內分組賽，保持手感，週六首戰...


In [4]:
#Part1: throw(here as the instruction replace) the trash and split the sentences in each paragraphs.
#PS. You can define "trash" by yourself, e.g. for me, →●▲ are trash.
df_read = df_read.replace(" ","")
df_read = df_read.replace("→","")
df_read = df_read.replace("●","")
df_read = df_read.replace("▲","")
df_read = df_read.replace("◎","")
df_read = df_read.replace("","")
df_read = df_read.replace("","")
df_read = df_read.replace("★","")
df_read = df_read.replace("◆","")
df_read = df_read.replace("☆","")
df_read = df_read.replace("\n","")
df_read = re.split(r'[，。?!,.;；]',df_read['context'][0])
#display(df_read)

In [5]:
#Part2: do zero padding on words of each sentences no matter the number of it is smaller than 20 or not.
#Here we add 50 words each sentences, and you need to add more than 30 at least.
#(for the reason that tranformimg with BERT can cause problem as "0 0 0" to "000".)
for j in range( len( df_read ) ):
    s=''
    for k in range( len( df_read[j] ) ):
        s+= df_read[j][k]
    for m in range( 30 ):
        s+=' 0'
    df_read[j] = s

In [6]:
#Part4: do zero padding on paragraphs no matter whether it's number of sentences are smaller than 30.
sen_30zero='0'+' 0'*30
l=[]
for j in range( len( df_read ) ):
    l.append( df_read[j] )
for j in range( 30-len( df_read ) ):
    l.append( sen_30zero )
df_read = l

In [7]:
#Part3: to use BERT to do word embedding, we transfrom myinput.csv to myinput.txt line by line,
#transform myinput.txt to mytmpfile.jsonl,
#and at the end output my(word embedding about your news).

input_filename = 'myinput.txt' #your news(input file) name
output_filename = 'myoutput' #your output file name

bertfile =  open(input_filename, "w")
for i in range(len(df_read)):
    bertfile.writelines(str(df_read[i]))
    bertfile.writelines('\n')
bertfile.close()

#The word embedding information will start as INFO:tensorflow:*** Example ***(it must be seen!!!)
!python ./extract_features.py \
    --input_file=./myinput.txt \
    --output_file=./mytmpfile.jsonl \
    --vocab_file=./chinese_L-12_H-768_A-12/vocab.txt \
    --bert_config_file=./chinese_L-12_H-768_A-12/bert_config.json \
    --init_checkpoint=./chinese_L-12_H-768_A-12/bert_model.ckpt \
    --layers=-1 \
    --max_seq_length=20 \
    --batch_size=8

/home/k66/anaconda3/envs/Lana/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/k66/anaconda3/envs/Lana/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/k66/anaconda3/envs/Lana/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/k66/anaconda3/envs/Lana/lib/python3.6/site-packages/ten


W1010 19:57:02.853961 139879124784960 deprecation_wrapper.py:119] From /home/k66/桌面/predict_news/modeling.py:490: The name tf.assert_less_equal is deprecated. Please use tf.compat.v1.assert_less_equal instead.

Instructions for updating:
Use keras.layers.dense instead.
W1010 19:57:02.892555 139879124784960 deprecation.py:323] From /home/k66/桌面/predict_news/modeling.py:671: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W1010 19:57:02.970713 139879124784960 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f37bf469438>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f

W1010 19:57:03.591745 139879124784960 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f37bf356be0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f37bf356be0>>: AssertionError: Bad argument number for Name: 3, expecting 4
W1010 19:57:03.678930 139879124784960 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f37bf3b3390>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object 

W1010 19:57:04.245766 139879124784960 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f37bf14ac50>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f37bf14ac50>>: AssertionError: Bad argument number for Name: 3, expecting 4
W1010 19:57:04.325735 139879124784960 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f37bf3c3e10>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object 

W1010 19:57:05.003598 139879124784960 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f37bee7c438>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f37bee7c438>>: AssertionError: Bad argument number for Name: 3, expecting 4
W1010 19:57:05.082825 139879124784960 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f37bee7c438>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object 

...

W1010 19:57:07.665049 139879124784960 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f37bf377278>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f37bf377278>>: AssertionError: Bad argument number for Name: 3, expecting 4
W1010 19:57:07.752447 139879124784960 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f37be760ba8>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object 

W1010 19:57:08.317751 139879124784960 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f37bf17def0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f37bf17def0>>: AssertionError: Bad argument number for Name: 3, expecting 4
W1010 19:57:08.394351 139879124784960 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f37be75c4a8>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object 

W1010 19:57:09.002214 139879124784960 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f37c0045518>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f37c0045518>>: AssertionError: Bad argument number for Name: 3, expecting 4

W1010 19:57:09.004262 139879124784960 deprecation_wrapper.py:119] From ./extract_features.py:171: The name tf.trainable_variables is deprecated. Please use tf.compat.v1.trainable_variables instead.


W1010 19:57:09.006186 139879124784960 deprecation_wrapper.py:119] From ./extract_features.py:184: The name tf.train.init_from_checkpoint is deprecated. Please use tf.compat.v1.train.init_from_checkpoint instead.

INFO:tensorflow:**** Trainable Variables ****

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1010 19:57:09.608289 139879124784960 deprecation.py:323] From /home/k66/anaconda3/envs/Lana/lib/python3.6/site-packages/tensorflow/python/ops/array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
I1010 19:57:09.835828 139879124784960 monitored_session.py:240] Graph was finalized.
2019-10-10 19:57:09.836148: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2019-10-10 19:57:09.858079: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3591655000 Hz
2019-10-10 19:57:09.858480: I tensorflow/compiler/xla/service/service.cc:168] XLA service

2019-10-10 19:57:10.340368: W tensorflow/core/framework/op_kernel.cc:1502] OP_REQUIRES failed at save_restore_v2_ops.cc:184 : Not found: ./chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001; No such file or directory
2019-10-10 19:57:10.341091: W tensorflow/core/framework/op_kernel.cc:1502] OP_REQUIRES failed at save_restore_v2_ops.cc:184 : Not found: ./chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001; No such file or directory
2019-10-10 19:57:10.341290: W tensorflow/core/framework/op_kernel.cc:1502] OP_REQUIRES failed at save_restore_v2_ops.cc:184 : Not found: ./chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001; No such file or directory
2019-10-10 19:57:10.341447: W tensorflow/core/framework/op_kernel.cc:1502] OP_REQUIRES failed at save_restore_v2_ops.cc:184 : Not found: ./chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001; No such file or directory
2019-10-10 19:57:10.341523: W tensorflow/core/framework/op_kernel.cc:1502] OP_REQUIRES failed at

In [8]:
df = pd.read_json('mytmpfile.jsonl',lines=True, orient='records')
a_sentence=[]
para=[]
for linex_index in range(30):
    a_sentence=[]
    for seq in range(20):
        a_sentence.append( df['features'][linex_index][seq]['layers'][0]['values'] )
    para.append(a_sentence)
np.save('embedding/'+output_filename+'.npy',para)
